In [61]:
import pandas as pd
import os

In [62]:
TARGET_NAMES = ['toxic', 'severe_toxic', 'obscene', 'threat',
                        'insult', 'identity_hate']

In [63]:
def read_dir(dir_path):
    files = os.listdir(dir_path)
    dfs = [pd.read_csv(os.path.join(dir_path, file_path)) for file_path in files]
    
    return dfs
    
    

In [69]:
dfs = read_dir('../enesmbles/w2v_ensemble_cbow/')

In [70]:
def simple_ensemble(dfs):
    ensemble = dfs.pop()
    
    for df in dfs:
        ensemble = ensemble.loc[:, TARGET_NAMES].add(df.loc[:, TARGET_NAMES])
        
    ensemble = ensemble / (len(dfs) + 1)
    ensemble[ensemble.loc[:, TARGET_NAMES] > 1] = 1
    ensemble[ensemble.loc[:, TARGET_NAMES] < 0] = 0
    
    return ensemble

In [71]:
ensemble = simple_ensemble(dfs)

In [72]:
ensemble_df = pd.concat([dfs[0].id.to_frame(), ensemble], axis=1)

In [73]:
ensemble_df.to_csv('w2v_ensemble.csv', index=False)